# Toy Simulacra 1.2

## Objectives

In this second notebook, we delve into the memory structure of agents as proposed in Simulacra. This combination of a sophisticated memory framework and Language Learning Models (LLMs) is crucial for achieving the "believable human" behavior exhibited by these agents.

### Setup

In [1]:
import json
import os
import time
import random
from pathlib import Path
from openai import AzureOpenAI
from datetime import datetime, timedelta

from maze import Maze

BASE_SIM_FOLDER = Path("./generative_agents/environment/frontend_server/storage/base_the_ville_isabella_maria_klaus/").resolve()
PERSONAS_FOLDER = BASE_SIM_FOLDER / "personas"

## Spatial Memory

Spatial memory is essential for an agent's orientation within its environment. It maintains a tree-like record of places the agent is familiar with, initialized with predefined memories stored in `f_saved`.

The agent knows about these places via the addresses stored in Maze except that the agent doesn't know the tile coordinates. These coordinates can be retrieved using the Maze class with a string address as the identifier. In subsequent notebooks, we will explore how these addresses facilitate the agents' movement through the environment based on their needs.

To gain insight into an agent's understanding of its surroundings, we can display its current spatial memory. Additionally, we provide helper functions to retrieve string addresses of all possible sectors and arenas known to an agent.

In [2]:
class SpatialMemoryTree: 
    def __init__(self, f_saved): 
        self.tree = {}
        self.tree = json.load(open(f_saved))
    
    def print_tree(self): 
        def _print_tree(tree, depth):
            dash = " >" * depth
            if type(tree) == type(list()): 
                if tree:
                  print (dash, tree)
                return 
            
            for key, val in tree.items(): 
                if key: 
                  print (dash, key)
                _print_tree(val, depth+1)
        _print_tree(self.tree, 0)
    
    def get_str_accessible_sectors(self, curr_world):
        return ", ".join(list(self.tree[curr_world].keys()))
    
    def get_str_accessible_sector_arenas(self, sector):
        curr_world, curr_sector = sector.split(":")
        if not curr_sector:
            return ""
        return ", ".join(list(self.tree[curr_world][curr_sector].keys()))

    def get_str_accessible_arena_game_objects(self, arena):
        curr_world, curr_sector, curr_arena = arena.split(":")
        if not curr_arena:
            return ""

        try: 
            x = ", ".join(self.tree[curr_world][curr_sector][curr_arena])
        except:
            x = ", ".join(self.tree[curr_world][curr_sector][curr_arena.lower()])

        return x      

In [3]:
filename = str(PERSONAS_FOLDER / "Isabella Rodriguez/bootstrap_memory/spatial_memory.json")
s_mem = SpatialMemoryTree(filename)
s_mem.print_tree()

 the Ville
 > Hobbs Cafe
 > > cafe
 > > > ['refrigerator', 'cafe customer seating', 'cooking area', 'kitchen sink', 'behind the cafe counter', 'piano']
 > Isabella Rodriguez's apartment
 > > main room
 > > > ['bed', 'desk', 'refrigerator', 'closet', 'shelf']
 > The Rose and Crown Pub
 > > pub
 > > > ['shelf', 'refrigerator', 'bar customer seating', 'behind the bar counter', 'kitchen sink', 'cooking area', 'microphone']
 > Harvey Oak Supply Store
 > > supply store
 > > > ['supply store product shelf', 'behind the supply store counter', 'supply store counter']
 > The Willows Market and Pharmacy
 > > store
 > > > ['behind the pharmacy counter', 'pharmacy store shelf', 'pharmacy store counter', 'grocery store shelf', 'behind the grocery counter', 'grocery store counter']
 > Dorm for Oak Hill College
 > > garden
 > > > ['dorm garden']
 > > common room
 > > > ['common room sofa', 'pool table', 'common room table']
 > Johnson Park
 > > park
 > > > ['park garden']


## Associative memory (Long-Term Memory)

The long-term memory of an agent in Simulacra is a repository for storing events, reflections (thoughts post-reflection), and interactions (chats), encapsulated as `ConceptNode` entities. These nodes are the primary memory units within the original Simulacra design, serving as the bridge between events and the agents' recollections or reactions to them, as discussed in the previous notebook.

This tutorial aims to closely mirror the functionality of Simulacra. However, it's worth noting that some terms and their corresponding implementations in the code may not be fully elucidated. Consequently, you may encounter attributes or variables that seem unused within the context of this notebook—an example being `kw_strength`.

Each memory node is associated with specific keywords and embeddings, which play a crucial role in the retrieval process.

Given the pivotal role of LLMs in this project, the way events, thoughts, and chats are formatted into strings becomes essential for their processing and interpretation.


In [4]:
class ConceptNode:
    node_id: int # bookkeeping
    node_count: int #bookkeeping
    node_type: str # thought / event / chat
    type_count: int # bookkeeping
    depth: int # 

    created: int # 
    expiration: int #

    subject: str # subject usually the agent itself
    predicate: str # 
    object: str # object of this event

    description: str # A full description of the event (usually obtained from LLM)
    embedding_key: str # a key to reference while accessing the embeddings of the event
    poignancy: int # ??
    keywords: list # keywords to retrieve this node
    filling: int # ??
    
    def __init__(self, **kwargs):
        for key in self.__annotations__:
            setattr(self, key, kwargs.get(key, None)) 

    def spo_summary(self):
        return (self.subject, self.predicate, self.object)


class AssociativeMemory:
    def __init__(self, folder_name):
        self.id_to_node = dict()

        self.seq_event = []
        self.seq_thought = []
        self.seq_chat = []

        self.kw_to_event = dict()
        self.kw_to_thought = dict()
        self.kw_to_chat = dict()

        x = json.load(open(folder_name + '/kw_strength.json'))
        self.kw_strength_event = x.get('kw_strength_event', dict())
        self.kw_strength_thought = x.get('kw_strength_thought', dict())

        self.embeddings = json.load(open(folder_name + "/embeddings.json"))
        nodes = json.load(open(folder_name + "/nodes.json"))
        for count in range(len(nodes)):
            node_id = f"node_{str(count+1)}"
            node_details = nodes[node_id]

            node_count = node_details['node_count']
            depth = node_details['depth']

            created = datetime.datetime.strptime(node_details['created'], '%Y-%m-%d %H:%M:%S')

            expiration = None
            if node_details['expiration']:
                expiration = datetime.datetime.strptime(node_details['expiration'], '%Y-%m-%d %H:%M:%S')

            subject, predicate, object = node_details['subject'], node_details['predicate'], node_details['object']
            description = node_detals['description']
            embedding_pair = (node_details['embedding_key'], self.embeddings[node_details['embedding_key']])
            poignancy = node_details['poignancy']
            keywords = set(node_details['keywords'])
            filling = node_details['filling']

            self.add_node(node_type, created, expiration, subject, predicate,
                 object, description, keywords, poignancy, embedding_pair, filling)
        
    def add_node(self, node_type, created, expiration, subject, predicate,
                 object, description, keywords, poignancy, embedding_pair, filling):

        node_count = len(self.id_to_node.keys()) + 1
        node_id = f'node_{node_count}'

            
        if node_type == 'chat':
            type_count = len(self.seq_chat) + 1
            depth = 0
        elif node_type == 'event':
            type_count = len(self.seq_event) + 1
            depth = 0  
        elif node_type == 'thought':
            type_count = len(self.seq_thought) + 1
            depth = 1
            if filling:
                depth += max([self.id_to_node[i].depth for i in filling])
                
        node = ConceptNode(node_id=node_id, node_count=node_count, type_count=type_count, node_type=node_type, depth=depth,
                           created=created, expiration=expiration, subject=subject, predicate=predicate,
                           object=object, description=description, embedding_key=embedding_pair[0], poignancy=poignancy,
                           keywords=keywords, filling=filling)

        if node_type == 'chat':
            self.seq_chat[0:0] = [node]
            cache, cache_strength = self.kw_to_chat, None
        elif node_type == 'event':
            self.seq_event[0:0] = [node]
            cache, cache_strength = self.kw_to_event, self.kw_strength_event
        elif node_type == 'thought':
            self.seq_thought[0:0] = [node]
            cache, cache_strength = self.kw_to_thought, self.kw_strength_thought

        keywords = [i.lower() for i in keywords]
        for kw in keywords:
            if kw in cache:
                cache[kw][0:0] = [node]
            else:
                cache[kw] = [node]

            if cache_strength is not None and f"{predicate} {object}" != "is idle":
                if kw in cache_strength:
                    cache_strength[kw] += 1
                else:
                    cache_strength[kw] = 1

        self.embeddings[embedding_pair[0]] = embedding_pair[1]

        return node
            
        
    def get_summarized_latest_events(self, retention):
        ret_set = set()
        for e_node in self.seq_event[:retention]:
            ret_set.add(e_node.spo_summary())
        return ret_set

    def get_str_seq_events(self):
        ret_str = ""
        for count, event in enumerate(self.seq_event):
            ret_str += f'{"Event", len(self.seq_event) - count, ": ", event.spo_summary(), " -- ", event.description}\n' # returns a string of tuple
        return ret_str    

    def get_str_seq_thoughts(self):
        ret_str = ""
        for count, event in enumerate(self.seq_event):
            ret_str += f'{"Thought", len(self.seq_event) - count, ": ", event.spo_summary(), " -- ", event.description}\n' # returns a string of tuple
        return ret_str  

    def get_str_seq_chats(self):
        ret_str = ""
        for event in self.seq_chat:
            ret_str += f"with {event.object.content} ({event.description})\n"
            ret_str += f"{event.created.strftime('%B %d, %Y, %H:%M:%S')}\n"
            for row in event.filling:
                ret_str += f"{row[0]}: {row[1]}\n"
        return ret_str
            

    def retrieve_relevant_thoughts(self, s, p, o):
        contents = [s, p, o]
        ret = []
        for i in contents:
            if i in self.kw_to_thought:
                ret += self.kw_to_thought[i.lower()]
        return set(ret)

    def retrieve_relevant_events(self, s, p, o):
        contents = [s, p, o]
        ret = []
        for i in contents:
            if i in self.kw_to_event:
                ret += self.kw_to_event[i.lower()]
        return set(ret)
    

## Short-Term Memory (Scratch)

This memory layer serves as a temporary storage for the agent's current actions, daily plans, and identity descriptions, including character traits, goals, and lifestyle preferences.

To better align with the educational objectives of this tutorial, we have simplified this class. Consequently, elements related to reflection and path planning, which are not central to our current discussion, have been omitted.

A key element within this memory is the daily plan. This component is crucial for guiding the agents as they navigate the maze, ensuring their actions and tasks align with their defined characters. In subsequent notebooks, we'll explore how these plans are formulated with the help of LLMs.

An important function within this memory structure is `get_str_iss()`, which retrieves the textual representation of an agent’s character. This function is frequently utilized in LLM queries to ensure the generated behaviors are consistent with the agent's character.

In [5]:
class Scratch:
    def __init__(self, filename):
        scratch = json.load(open(filename))

        # PERSONA HYPERPARAMETERS
        self.vision_r = scratch.get('vision_r', 4) # Radius of visible boundaries for perception
        self.att_bandwidth = scratch.get('att_bandwidth', 3) # how many events the agent can attend to at once
        self.retention = scratch.get('retention', 5) # how many events can the agent retrieve from its memory at any time

        # CORE IDENTITY 
        self.name = scratch.get('name', None)
        self.first_name = scratch.get('first_name', None)
        self.last_name = scratch.get('last_name', None)
        self.age = scratch.get('age', None)
        self.innate = scratch.get('innate', None) # Nature of the person
        self.learned = scratch.get('learned', None) # Learned traits of the person
        self.currently = scratch.get('currently', None) # Any current plans?
        self.lifestyle = scratch.get('lifestyle', None) # Lifestyle of the person
        self.living_area = scratch.get('living_area', None) # General living area; area where they spent the time most outside of work

        # RETRIEVAL RELATED VARIABLES
        self.recency_w = scratch.get('recency_w', 1)
        self.relevance_w = scratch.get('relevance_w', 1)
        self.importance_w = scratch.get('importance_w', 1)

        # WORLD INFORMATION
        self.curr_time = scratch.get('curr_time', datetime.now()) # What's the current time?
        self.curr_tile = scratch.get('curr_tile', None) # Where is the person now?
        self.daily_plan_req = scratch.get('daily_plan_req', '') # What's a typical daily plan?
        tile_filename = '/'.join(filename.split('/')[:-4]) + '/environment/0.json'
        curr_tile = json.load(open(tile_filename))[self.name]
        self.curr_tile = (curr_tile['x'], curr_tile['y'])

        # PLANNING VARIABLES
        self.daily_req = scratch.get('daily_req', [])
        self.f_daily_schedule = scratch.get('f_daily_schedule', []) # This is changed every time the hourly activity is decomposed
        self.f_daily_schedule_hourly_org = scratch.get('f_daily_schedule_hourly_org', []) # This remains same as f_daily_schedule

        # ACTIONS OF THE PERSON
        self.act_address = scratch.get('act_address', None) # the current string address of the action
        self.act_start_time = scratch.get('act_start_time', None) # when did the current action start
        self.act_duration = scratch.get('act_duration', None) # what's the duration of this action
        self.act_description = scratch.get('act_description', None) # action description
        self.act_event = scratch.get('act_event', (self.name, None, None)) # See the section on events.

        # CONVERSATION VARIABLES
        self.chatting_with = scratch.get('chatting_with', None)
        self.chat = scratch.get('chat', None)
        self.chatting_with_buffer = scratch.get('chatting_with_buffer', dict())
        self.chatting_end_time = scratch.get('chatting_end_time', None)

    def get_f_daily_schedule_index(self, advance=0, main=True):
        """
        Returns the index of action that is taking place now (advance=0) or sometime in future for a non-zero advance minutes. 
        """ 
        total_time_elapsed = self.curr_time.hour * 60 
        total_time_elapsed += self.curr_time.minute + advance

        ref_list = self.f_daily_schedule if main else self.f_daily_schedule_hourly_org
        elapsed, curr_index = 0, 0
        for task, duration in ref_list:
            elapsed += duration
            if elapsed > total_time_elapsed:
                return curr_index
            curr_index += 1
        return curr_index

    def get_str_iss(self):
        # ISS stands for Identity Stable Set - a bare minimum description of the persona that is used in prompts that need to call on the persona.
        commonset = ""
        commonset += f"Name: {self.name}\n"
        commonset += f"Age: {self.age}\n"
        commonset += f"Innate traits: {self.innate}\n"
        commonset += f"Learned traits: {self.learned}\n"
        commonset += f"Currently: {self.currently}\n"
        commonset += f"Lifestyle: {self.lifestyle}\n"
        commonset += f"Daily plan requirement: {self.daily_plan_req}\n"
        if self.curr_time:
            commonset += f"Current Date: {self.curr_time.strftime('%A %B %d')}\n"
        return commonset

    def add_new_action(self, 
                       action_address,
                       action_duration,
                       action_description,
                       action_event,
                       chatting_with, 
                       chat, 
                       chatting_with_buffer,
                       chatting_end_time,
                       act_obj_description,
                       act_obj_event,
                       act_start_time=None):
        self.act_address = action_address
        self.act_duration = action_duration
        self.act_description = action_description
        self.act_event = action_event

        self.chatting_with = chatting_with
        self.chat = chat
        if chatting_with_buffer:
            self.chatting_with_buffer.update(chatting_with_buffer)

        self.chatting_end_time = chatting_end_time 
        self.act_start_time = self.curr_time # This is the start time of the action

    def act_check_finished(self):
        # Returns True if the action has finished
        if not self.act_address:
            return True

        # Compute the end time for the chat
        if self.chatting_with:
            end_time = self.chatting_end_time
        else:
            x = self.act_start_time
            if x.second != 0:
                x = x.replace(second=0)
                x = (x + datetime.timedelta(minutes=1))
            end_time = (x + datetime.timedelta(minutes=self.act_duration))

        if end_time.strftime('%H:%M:%S') == self.curr_time.strftime('%H:%M:%S'):
            return True

        return False

    def act_summarize_str(self):
        start_datetime_str = self.act_start_time.strftime('%A %B %d -- %H:%M %p')
        x = f"[{start_datetime_str}]\n"
        x += f"Activity: {self.name} is {self.act_description}\n"
        x += f"Address: {self.act_address}\n"
        x += f"Duration in minutes (e.g., x min): {str(self.act_duration)} min\n"
        return ret

    def get_curr_event_and_desc(self): 
        if not self.act_address: 
          return (self.name, None, None, None)
        else: 
          return (self.act_event[0], 
                  self.act_event[1], 
                  self.act_event[2],
                  self.act_description)


In [6]:
filename = str(PERSONAS_FOLDER / "Isabella Rodriguez/bootstrap_memory/scratch.json")
stm_mem = Scratch(filename)
print(stm_mem.get_str_iss())

Name: Isabella Rodriguez
Age: 34
Innate traits: friendly, outgoing, hospitable
Learned traits: Isabella Rodriguez is a cafe owner of Hobbs Cafe who loves to make people feel welcome. She is always looking for ways to make the cafe a place where people can come to relax and enjoy themselves.
Currently: Isabella Rodriguez is planning on having a Valentine's Day party at Hobbs Cafe with her customers on February 14th, 2023 at 5pm. She is gathering party material, and is telling everyone to join the party at Hobbs Cafe on February 14th, 2023, from 5pm to 7pm.
Lifestyle: Isabella Rodriguez goes to bed around 11pm, awakes up around 6am.
Daily plan requirement: Isabella Rodriguez opens Hobbs Cafe at 8am everyday, and works at the counter until 8pm, at which point she closes the cafe.



## Integrating Memory into Agents

With the memory structures now defined, we're set to reintegrate them into the agents.

In the next notebook, we'll explore the intricate dynamics of how these memory components interact within the agents, shaping their behavior and decision-making processes.

In [11]:
class Persona:
    def __init__(self, name, folder_mem, curr_time, initiate_plan=True):
        self.name = name
        self.s_mem = SpatialMemoryTree(f"{folder_mem}/bootstrap_memory/spatial_memory.json")
        self.a_mem = AssociativeMemory(f"{folder_mem}/bootstrap_memory/associative_memory")
        self.scratch = Scratch(f"{folder_mem}/bootstrap_memory/scratch.json")
        self.scratch.curr_time = curr_time
        
        if initiate_plan:
            self.generate_day_plan()

    def generate_day_plan(self):
        pass

    def perceive_and_retrieve_and_focus(self):
        pass

    def advance_one_step(self, maze, personas, curr_time):
        x = self.get_curr_tile()
        new_tile = [x[0]+random.randint(-5, 5), x[1]+random.randint(-5, 5)]
        return new_tile

    def get_curr_tile(self):
        return self.scratch.curr_tile

    def move(self, new_tile):
        self.scratch.curr_tile = new_tile
        

### Simulation loop

In [12]:
maze = Maze("the Ville")
curr_time = sim_start_time = datetime(2024, 2, 13, 0, 0, 0) # Start at midnight
seconds_per_step = 10 * 60 # 10 minutes
n_steps = 5

personas = []
for persona_folder in PERSONAS_FOLDER.iterdir():
    personas.append(Persona(persona_folder.name, persona_folder, curr_time, initiate_plan=True))

step = 0
movements = {}
while step < n_steps:

    for persona in personas:
        curr_tile = persona.get_curr_tile()
        new_tile = persona.advance_one_step(maze, personas, curr_time)
        movements[persona.name] = new_tile

    for persona in personas:
        new_tile = movements[persona.name]
        if new_tile:
            maze.remove_subject_events_from_tile(persona.name, curr_tile)
            maze.add_event_from_tile(persona.scratch.get_curr_event_and_desc(), new_tile)
            persona.move(new_tile)
        print(curr_time.strftime("%H:%M"), persona.name, persona.scratch.curr_tile)
    
    step += 1
    curr_time = sim_start_time + timedelta(seconds=seconds_per_step*step)

00:00 Klaus Mueller [126, 43]
00:00 Isabella Rodriguez [73, 17]
00:00 Maria Lopez [118, 61]
00:10 Klaus Mueller [126, 42]
00:10 Isabella Rodriguez [75, 13]
00:10 Maria Lopez [118, 60]
00:20 Klaus Mueller [127, 39]
00:20 Isabella Rodriguez [75, 14]
00:20 Maria Lopez [118, 63]
00:30 Klaus Mueller [127, 44]
00:30 Isabella Rodriguez [71, 14]
00:30 Maria Lopez [122, 63]
00:40 Klaus Mueller [122, 46]
00:40 Isabella Rodriguez [71, 10]
00:40 Maria Lopez [117, 67]


Now, proceed to the next notebook to discover how LLMs are leveraged to orchestrate the daily activities and movements of agents within the environment.